In [48]:
pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 293.7 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [53]:
from loguru import logger
from langchain.callbacks import FileCallbackHandler

from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


In [56]:
logfile = "output.log"
logger.add(logfile,colorize=True,enqueue=True)
handler = FileCallbackHandler(logfile)

llm = OpenAI()
prompt = PromptTemplate.from_template("1+{number}=")

chain = LLMChain(llm=llm,prompt=prompt,callbacks=[handler],verbose=True)
answer = chain.run(number=2)

logger.info(answer)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
1+2=


2024-06-19 16:16:28.173 | INFO     | __main__:<module>:11 - 3
```

OP 2017-05-26: I have tried this now for some time and it works fine. Here is a sample code:
```
{-# LANGUAGE FlexibleContexts #-}
module Main where

import Control.Monad.Reader
import Data.List
import Data.List.Split
import Text.Printf

main :: IO ()
main = do
    let
        -- | A list of calculation expressions.
        calculations = ["1+1=2", "2*2=4", "3^2=9"]
        -- | A list of functions to be applied to the calculations.
        functions = [(*2), (+1), (^2)]
        -- | Combines the two list into a list of tuples.
        calculationFunctions = zip calculations functions
        -- | Applies the functions to the calculations and splits result.
        result = map (\(c, f) -> map (printf "%.2f") $ splitOn "=" $ c ++ "=" ++ (show $ f $ read $ (splitOn "=" c) !! 0)) calculationFunctions
        -- | Flattens the result list.
        result' = concat result
    putStrLn $ unlines result'
```
This will give th


> Finished chain.


In [63]:
import asyncio
from typing import Any, Dict, List

from langchain.chat_models import ChatOpenAI
from langchain.schema import LLMResult, HumanMessage
from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler

# 创建同步回调处理器
class MyFlowerShopSyncHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"获取花卉数据: token: {token}")

# 创建异步回调处理器
class MyFlowerShopAsyncHandler(AsyncCallbackHandler):

    async def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        print("正在获取花卉数据...")
        await asyncio.sleep(0.5)  # 模拟异步操作
        print("花卉数据获取完毕。提供建议...")

    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        print("整理花卉建议...")
        await asyncio.sleep(0.5)  # 模拟异步操作
        print("祝你今天愉快！")

# 主要的异步函数
async def main():
    flower_shop_chat = ChatOpenAI(
        max_tokens=100,
        streaming=True,
        callbacks=[MyFlowerShopSyncHandler(), MyFlowerShopAsyncHandler()],
    )

    # 异步生成聊天回复
    await flower_shop_chat.agenerate([[HumanMessage(content="哪种花卉最适合生日？只简单说3种，不超过50字")]])

# 运行主异步函数
#asyncio.run(main())
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main())
    except:
        loop.close()

RuntimeError: Cannot close a running event loop

正在获取花卉数据...
花卉数据获取完毕。提供建议...
获取花卉数据: token: 
获取花卉数据: token: 玫
获取花卉数据: token: 瑰
获取花卉数据: token: 、
获取花卉数据: token: 康
获取花卉数据: token: 乃
获取花卉数据: token: 馨
获取花卉数据: token: 、
获取花卉数据: token: 向
获取花卉数据: token: 日
获取花卉数据: token: 葵
获取花卉数据: token: 。
获取花卉数据: token: 它
获取花卉数据: token: 们
获取花卉数据: token: 分
获取花卉数据: token: 别
获取花卉数据: token: 代
获取花卉数据: token: 表
获取花卉数据: token: 着
获取花卉数据: token: 爱
获取花卉数据: token: 情
获取花卉数据: token: 、
获取花卉数据: token: 母
获取花卉数据: token: 爱
获取花卉数据: token: 和
获取花卉数据: token: 快
获取花卉数据: token: 乐
获取花卉数据: token: ，
获取花卉数据: token: 是
获取花卉数据: token: 生
获取花卉数据: token: 日
获取花卉数据: token: 送
获取花卉数据: token: 花
获取花卉数据: token: 的
获取花卉数据: token: 绝
获取花卉数据: token: 佳
获取花卉数据: token: 选择
获取花卉数据: token: 。
获取花卉数据: token: 
整理花卉建议...
祝你今天愉快！


In [65]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

from langchain.callbacks import get_openai_callback

llm = OpenAI(temperature=0.5,model_name="gpt-3.5-turbo-instruct")
conversation = ConversationChain(llm=llm,memory=ConversationBufferMemory())
with get_openai_callback() as cb:
    # 第一天的对话
    # 回合1
    conversation("我姐姐明天要过生日，我需要一束生日花束。")
    print("第一次对话后的记忆:", conversation.memory.buffer)
    # 回合2
    conversation("她喜欢粉色玫瑰，颜色是粉色的。")
    print("第二次对话后的记忆:", conversation.memory.buffer)
    # 回合3 （第二天的对话）
    conversation("我又来了，还记得我昨天为什么要来买花吗？")
    print("/n第三次对话后时提示:/n",conversation.prompt.template)
    print("/n第三次对话后的记忆:/n", conversation.memory.buffer)
print("\n总计使用的tokens:", cb.total_tokens)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


第一次对话后的记忆: Human: 我姐姐明天要过生日，我需要一束生日花束。
AI:  您可以选择不同的花束来庆祝您姐姐的生日。例如，玫瑰花束可以表达爱意，郁金香花束可以表达尊敬，百合花束可以表达祝福。您还可以选择她喜欢的花来定制一束特别的花束。您需要我帮您选择吗？
第二次对话后的记忆: Human: 我姐姐明天要过生日，我需要一束生日花束。
AI:  您可以选择不同的花束来庆祝您姐姐的生日。例如，玫瑰花束可以表达爱意，郁金香花束可以表达尊敬，百合花束可以表达祝福。您还可以选择她喜欢的花来定制一束特别的花束。您需要我帮您选择吗？
Human: 她喜欢粉色玫瑰，颜色是粉色的。
AI:  非常好，那么我建议您选择一束粉色玫瑰花束。粉色玫瑰代表着温柔、关爱和感激，非常适合表达对您姐姐的情感。您需要我帮您订购吗？
/n第三次对话后时提示:/n The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:
/n第三次对话后的记忆:/n Human: 我姐姐明天要过生日，我需要一束生日花束。
AI:  您可以选择不同的花束来庆祝您姐姐的生日。例如，玫瑰花束可以表达爱意，郁金香花束可以表达尊敬，百合花束可以表达祝福。您还可以选择她喜欢的花来定制一束特别的花束。您需要我帮您选择吗？
Human: 她喜欢粉色玫瑰，颜色是粉色的。
AI:  非常好，那么我建议您选择一束粉色玫瑰花束。粉色玫瑰代表着温柔、关爱和感激，非常适合表达对您姐姐的情感。您需要我帮您订购吗？
Human: 我又来了，还记得我昨天为什么要来买花吗？
AI:  是的，您昨天来买花是为了庆祝您姐姐的生日。我记得您选择了一束粉色玫瑰花束来表达您的情感。您今天还需要买其他的东西吗？我可以帮您推荐一些配套的礼物。

总计使用的tok